<a href="https://colab.research.google.com/github/Arghya-Sengupta/Major-Project/blob/main/YOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# run this code to clone the yolov5 repository from github

!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.1-78-gc3d5ac1 torch 1.10.0+cu111 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.6/107.7 GB disk)


In [24]:
# if you are using google colab
# run this code to fetch data from google drive

from google.colab import drive
drive.mount('/content/drive')
!unzip -q /content/drive/MyDrive/Data/sample_data.zip -d /content/
!cp "/content/drive/MyDrive/Data/custom_coco.yaml" "/content/yolov5/data/"
print("Check for /content/yolov5/data/custom_coco.yaml")

Check for /content/yolov5/data/custom_coco.yaml


In [35]:
# code for modifying the bounding boxes in yolo format

import os
from PIL import Image

# modify the directories accordingly
datapath = '/content/sample_data'
labelpath = '/content/sample_data/labels/'
classfilename = '/content/drive/MyDrive/Data/food100.names'

def convert_yolo_bbox(img_size, box):
    # img_bbox file format is [0:img] [1:left X] [2:bottom Y] [3:right X] [4:top Y]
    dw = 1./img_size[0]
    dh = 1./img_size[1]
    x = (int(box[1]) + int(box[3]))/2.0
    y = (int(box[2]) + int(box[4]))/2.0
    w = abs(int(box[3]) - int(box[1]))
    h = abs(int(box[4]) - int(box[2]))
    x *= dw
    w *= dw
    y *= dh
    h *= dh
    # yolo_bbox file format is center x, y and width, height
    return (x,y,w,h)

def generate_bbox_file(classid):
    dataDir = os.path.join(datapath, str(classid))
    labelDir = os.path.join(labelpath, str(classid))
    bb_filename = os.path.join(dataDir, 'bb_info.txt')
    if not os.path.exists(labelDir):
        os.makedirs(labelDir)
    with open(bb_filename) as fp:
        for line in fp.readlines():
            # img_bbox file is [0:img] [1:left X] [2:bottom Y] [3:right X] [4:top Y]
            img_bbox = line.strip('\n').split(' ')
            if img_bbox[0] != 'img':
                image_filename = os.path.join(dataDir, img_bbox[0]+'.jpg')
                yolo_label_filename = os.path.join(labelDir, img_bbox[0]+'.txt')
                with open(yolo_label_filename, 'w') as f:
                    img = Image.open(image_filename)
                    yolo_bbox = convert_yolo_bbox(img.size, img_bbox)
                    if (yolo_bbox[2] > 1) or (yolo_bbox[3] > 1):
                        print("image %s bbox is " %(image_filename) + ' '.join(map(str, yolo_bbox)))
                    f.write(str(classid-1) + ' ' + ' '.join(map(str, yolo_bbox)) + '\n')
                    img.close()
                    f.close()
        fp.close()

classid = 0
classid2name = {}
if os.path.exists(classfilename):
    with open(classfilename) as cf:
        for line in cf.readlines():
            classname = line.strip('\n')
            classid = classid + 1
            classid2name[classid] = classname

for id in classid2name.keys():
    print("generating %d %s" %(id, classid2name[id]))
    generate_bbox_file(id)
print('Completed')

generating 1 rice
Completed


In [36]:
# delete the UECFOOD100 folder first (if present)
# %rm -rf /content/UECFOOD100
# code for spliting both images and labels for training and validation

import os
import glob
import shutil
from os.path import isfile, join
from random import randint
from os import listdir

directory = '/content/sample_data/'
nc = 1	# number of catagories

def make_directories():
	os.makedirs('/content/UECFOOD100/images/train')
	os.mkdir('/content/UECFOOD100/images/val')
	os.makedirs('/content/UECFOOD100/labels/train')
	os.mkdir('/content/UECFOOD100/labels/val')

def split_files(probability):
	t=0
	v=0
	for i in range(1,nc+1):
		new_dir = directory + str(i) + '/'
		for f in listdir(new_dir):
			if isfile(join(new_dir, f)) and f.endswith((".jpg", ".jpeg", ".png")):
				file_name = os.path.splitext(f)[0]
													
				if(randint(1, 100) <= probability):	
					split = 'train/'
					t +=1
				else:
					split = 'val/'
					v += 1

				imgage_src = new_dir + f
				imgage_dst = "/content/UECFOOD100/images/" + split

				label_src = directory + "labels/" + str(i) + "/" + file_name + ".txt"
				label_dst = "/content/UECFOOD100/labels/" + split

				shutil.copy2(imgage_src, imgage_dst)
				shutil.copy2(label_src, label_dst)
	print(t,"Images for training")
	print(v,"Images for validation")

make_directories()
split_files(70)	# 70% for training and 30% for validation

339 Images for training
161 Images for validation


In [37]:
# do with 60 epochs for best results

!python train.py --img 640 --batch 2 --epochs 1 --data /content/yolov5/data/custom_coco.yaml --weights yolov5s.pt --cache

# runs/train/exp/weights/best.pt
# yolov5x.pt

train: weights=yolov5s.pt, cfg=, data=/content/yolov5/data/custom_coco.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=1, batch_size=2, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 2 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-78-gc3d5ac1 torch 1.10.0+cu111 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015,

In [54]:
# detect a custom image
!python detect.py --weights /content/yolov5/runs/train/exp12/weights/best.pt --img 640 --conf 0.15 --source /content/test.jpg

detect: weights=['/content/yolov5/runs/train/exp12/weights/best.pt'], source=/content/test.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.15, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-78-gc3d5ac1 torch 1.10.0+cu111 CPU

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/test.jpg: 480x640 1 rice, Done. (0.257s)
Speed: 3.2ms pre-process, 256.6ms inference, 0.9ms NMS per image at shape (1, 3, 640, 640)
Images saved in runs/detect/exp11
Food names saved in ../result.txt


In [32]:
%rm -rf /content/sample_data
!unzip -q /content/drive/MyDrive/Data/sample_data.zip -d /content/

In [33]:
import os
from os.path import isfile, join
from os import listdir

directory = '/content/sample_data/'
nc = 1	# number of catagories0/labels/val')

def split_files():
	x=1
	for i in range(1,nc+1):
		new_dir = directory + '1/'
		for f in listdir(new_dir):
			if isfile(join(new_dir, f)) and f.endswith((".jpg")):
				os.rename(new_dir+f , new_dir+str(x)+".jpg")
				x += 1
split_files()

In [50]:
# with open('/content/result.txt', 'x') as f:
#     f.write('Here is a new text file!')

# with open('/content/result.txt', "w") as myfile:
#     myfile.write("newData2"+"\n")
#     myfile.write("newData3"+"\n")